In [137]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import json
import spacy as sp

nlp = sp.load("es_core_news_md")

In [145]:
df = pd.read_json("Data/train-esp.json")
df_cook = pd.read_csv("Data/intents_cook.csv")
df_cook_others = pd.read_csv('Data/intents_cook_others.csv')

print("Processing the Intents.....")
with open('Data/intents.json') as json_data:
    intents = json.load(json_data)

df_cook_others["label"] = df_cook_others.label.map({ 'cook': 1, 'others': 0})
df_cook["label"] = df_cook.label.map({ 'title': 2, 'instructions': 1, 'title_instructions': 0})
df_cook_others

Processing the Intents.....


,Unnamed: 0,text,label
0,0,"Que hacer con 1 1/2 tazas de Oporto leonado, 1...",1
1,1,"Quiero Tarta de queso de higos confitados, ave...",1
2,2,Como preparar Posole de pavo,1
3,3,Si tengo 1/4 de taza de yogur natural de leche...,1
4,4,Como preparar Tarta de nueces a la antigua usa...,1
...,...,...,...
6571,6571,😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘...,0
6572,6572,Muchos besos no puedo darlos pero si decirlo,0
6573,6573,Gracias mi amorrrr,0
6574,6574,Te quiero millones,0


In [29]:
# Definir los documentos
documents = ['hola, como estás!',
                'Gana dinero, gana por la casa.',
                'Llámame.',
                'Hola, Llámame mañana?']
# Importar el contador de vectorizacion e inicializarlo

count_vector = CountVectorizer()
# Visualizar del objeto'count_vector' que es una instancia de 'CountVectorizer()'
count_vector.fit(documents)
names = count_vector.get_feature_names()

doc_array = count_vector.transform(documents).toarray()
doc_array

frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

/home/david/bin/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,casa,como,dinero,estás,gana,hola,la,llámame,mañana,por
0,0,1,0,1,0,1,0,0,0,0
1,1,0,1,0,2,0,1,0,0,1
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,1,1,0


In [34]:
# Dividir el conjunto de datos de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_cook_varios['text'], df_cook_varios['label'], random_state=1)
print('Number of rows in the total set: {}'.format(df_cook_varios.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 6576
Number of rows in the training set: 4932
Number of rows in the test set: 1644


In [39]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()
# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)
# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

In [41]:

naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

In [44]:
predictions = naive_bayes.predict(testing_data)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [85]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9884428223844283
Precision score:  0.9938650306748467
Recall score:  0.994475138121547
F1 score:  0.9941699907947222


In [112]:
def cook_others(text):
    # Dividir el conjunto de datos de entrenamiento y test
    X_train, X_test, y_train, y_test = train_test_split(df_cook_others['text'], df_cook_others['label'], random_state=1)
    
    # Instantiate the CountVectorizer method
    count_vector = CountVectorizer()
    # Fit the training data and then return the matrix
    training_data = count_vector.fit_transform(X_train)
    # Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
    testing_data = count_vector.transform(X_test)
    text_data = count_vector.transform([text])

    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    
    return naive_bayes.predict(text_data)

In [113]:
def cook(text):
    # Dividir el conjunto de datos de entrenamiento y test
    X_train, X_test, y_train, y_test = train_test_split(df_cook['text'], df_cook['label'], random_state=1)
    
    # Instantiate the CountVectorizer method
    count_vector = CountVectorizer()
    # Fit the training data and then return the matrix
    training_data = count_vector.fit_transform(X_train)
    # Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
    testing_data = count_vector.transform(X_test)
    text_data = count_vector.transform([text])

    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    
    return naive_bayes.predict(text_data)

In [136]:
def others(text):
    x = []
    y_train = []
    for i in range(4):
    #     print(intents["intents"][i])
        label = intents["intents"][i]["tag"]
        for j in intents["intents"][i]["patterns"]:
            x.append(j)
            y_train.append(label)
            
    # Instantiate the CountVectorizer method
    count_vector = CountVectorizer()
    # Fit the training data and then return the matrix
    training_data = count_vector.fit_transform(x)
    text_data = count_vector.transform([text])

    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    
    return naive_bayes.predict(text_data)

In [159]:
def search(text, first, second):
    ma = -1000000
    sol = ""
    doc = nlp(text)
    doc = doc.similarity(text)
    return sol
        
def response_others(tag):
    for i in range(4):
    #     print(intents["intents"][i])
        label = intents["intents"][i]["tag"]
        if label == tag:
            return intents["intents"][i]["responses"][0]

In [160]:
def get_response(text):
    firstIntent = cook_others(text)[0]
    if firstIntent == 0:
        intentOthers = others(text)[0]
        return response_others(intentOthers)
    else:
        cook_intents = cook(text)
        if cook_intents == 1:
            return search(text, "ingredients", "instructions")

        if cook_intents == 2:
            return search(text, "ingredients", "title")

        if cook_intents == 0:
            return search(text, "title", "instructions")
    return firstIntent

get_response("Buenos dias")

AttributeError: 'str' object has no attribute 'vector_norm'